In [ ]:
import scriptFile as sf
import importlib as impL
import helperFuncs as funcH
impL.reload(sf)
impL.reload(funcH)
a, result_dict = sf.study_silhouette_analysis(fold_to_run='hgsk_256_11/20210221_hgsk_256_11_cKM256_e10_UMAPud256_un20', bef_aft="before")

In [ ]:
#np.savez('/home/doga/Desktop/a.npz', labels_sorted=labels_sorted, preds_sorted=preds_sorted, data_perc_vec=data_perc_vec, allow_pickles=True)
#np.savez('/home/doga/Desktop/b.npz', a=a, result_dict=result_dict, allow_pickles=True)